In [1]:
#=====================import======================
from menu import Menu
#=================================================

#===============Programme Principal===============
Menu()
#=================================================

=========[Scoreboard]=========
['DJ_Vince [cheateur]', '94', '8x15', '34']
['DJ_Vince [cheateur]', '95', '8x15', '34']
['DJ_Vince [cheateur]', '22', '8x15', '10']
['DJ_Vince [cheateur]', '22', '8x15', '10']
['DJ_Vince', '27', '8x15', '10']
['DJ_Vince', '7', '8x15', '4']
['azerty [cheateur]', '12', '8x15', '4']
['edrtyghuj', '10', '8x15', '4']
['rtyhj', '10', '8x15', '4']
['zertyjk', '22', '8x15', '4']
['vince', '7', '8x15', '4']
['qfghjk', '7', '8x15', '4']
['vince', '17', '8x15', '4']
['Youpi!', '9', '8x15', '4']
====[Scoreboard  LifeMode]====
['DJ_Vince', '1']
['DJ_Vince', '4']
=========[Scoreboard]=========
['DJ_Vince [cheateur]', '94', '8x15', '34']
['DJ_Vince [cheateur]', '95', '8x15', '34']
['DJ_Vince [cheateur]', '22', '8x15', '10']
['DJ_Vince [cheateur]', '22', '8x15', '10']
['DJ_Vince', '27', '8x15', '10']
['DJ_Vince', '7', '8x15', '4']
['azerty [cheateur]', '12', '8x15', '4']
['edrtyghuj', '10', '8x15', '4']
['rtyhj', '10', '8x15', '4']
['zertyjk', '22', '8x15', '4']
['vince'